# Profiling

## Why profile?

Profiling is the act of finding the demands placed on your computer when a piece of code is run and, preferably, localising the demands to specific sections of code. This is important when optimising as it allows you to find which parts of your code contribute to the demand you want to reduce and then target your optimisation efforts there.

For instance, if ```function_1``` takes 99% of the runtime of your code and ```function2``` takes the rest, there is no point in optimising ```function2``` as even reducing the runtime to zero for that function would not produce a noticeable impact on the runtime of the code as a whole. Instead, optimising ```function1``` should be the priority.

For this reason, when optimising a code, the first step is to profile it. This means the time you spend optimising your code will have most impact.

## Profiling Effectively

### Profiling Runs

In many cases, when profiling and optimising a large and complex code, it may not be practical to perform full-scale executions of the code. This may be because, for example, the code is designed to run on HPC resources but you're profiling on your desktop. You may need to profile a simplified or scaled-down problem.

In addition, it may be that a code may be used in a number of different ways, such as performing many different kinds of simulation or analysing many different kinds of data set. It may be that the code can be run with a multitude of different options that will use different parts of the code. It may be that you need to develop a suite of cases to profile to get a measure of how the code performs in different scenarios.

### Length of Profiling Runs

If you profile the same bit of code twice, you may get noticeably different execution times for a number of reasons. The shorter the profiling run, the larger the fractional difference between runs is likely to be. This means that a code needs to run for at least a few seconds for profiling to produce meaningful results.

However, running the code for a long time is not a good solution either. When optimising a code, you will typically want to profile it after each attempt to improve it, to see if the changes you made had the desired effect. This means you will want to profile the code many times. If each profiling run takes a long time, this will slow down your development cycle.

As a result, it's often desirable to have a profiling run that is as short as possible while still producing meaningful results. Often, between 10s and 1 minute is a reasonable compromise, although this may vary depending on the code.

### Profiling Effective - Takeaway

There isn't a simple answer to ensure you get an effective profile of your code as it may be very problem-specific. You should try to think about what your code is doing and what you want to profile carefully and develop a strategy. Then repeat the profiling a few times to ensure the results remain the same.

## Profiling Tools

There are a number of different profiling tools available for use in Python. In this notebook, we will look at a sample of them - favouring ones which are simple to implement in a Jupyter Notebook. However, this list is far from exhaustive.

The examples in this section relate to running this notebook in Colab. To run this notebook in Anaconda locally a different approach may be required in some cases. Many of the commands are also available from the command line when profiling a ```.py``` file. Again, a different approach will be required in this usage case.

## Run-time Profiling

Profiling the run-time of the code is probably one of the most important parts of profiling. In many cases, the amount of time the code takes to run is the most limiting factor when deciding if a particular simulation can be carried out on the available computational resources in a reasonable amount of time. There are several useful bits of useful information we can gather.

### Timing Code

We can manually time code using the ```time``` module. One simple way to use this module is to use the ```time.time()``` function to get the current time at the start and end of the code you want to time. This value is returned as a ```float``` representing the number of seconds since the start of the Unix epoch, which began on 1 January 1970. ```time``` is part of the standard library, so no installation is required.

In [ ]:
import time

# Get the time at the start of the run
start_time = time.time()

# This is the code we want to time
total = 0
for i in range(1000000):
    total = total + i
print(total)

# Get the time at the end of the run
end_time = time.time()

# Calculate the total time of the run
total_time = end_time - start_time
print(total_time)

The ```time``` module is very limited for profiling, however, as it involves modifying the source code manually, which is time-consuming and introduces a risk of accidentally leaving timing code in the final version of the code, or introducing bugs when adding or removing timing code. It also doesn't give a detailed breakdown of where the time is spent. 

### Profiling By Function
Profiling by function gives us a high-level idea of how often functions are called and how long those calls last. One way to do this is to import the ```cProfile``` module and run a function using the ```cProfile.run()``` function, providing a string argument which is the command used to invoke the function. ```cProfile``` is part of the Python standard library and so is available without installing any additional packages.

In [ ]:
# Import the cProfile module
import cProfile

def is_even(value):
  if value%2 == 0:
    return True
  else:
    return False

def halve(value):
  return value / 2

def function_to_test(upper_value):
  result = 0
  for i in range(int(upper_value) +1):
    if is_even(i):
      result=result + halve(i)

  print(result)

# Call the cProfile.run() method with a string argument that is the call to the function you want to profile
cProfile.run('function_to_test(1e7)')

The results show the total time spent running the code and the total number of function calls. Then, for each function, it shows:
* ```ncalls```: the number of times the function was called.
* ```tottime```: the total time spent in the function, excluding time spent in functions called by the function.
* ```percall```: the time spent in the function per call, excluding time spent in functions called by the function.
* ```cumtime```: the total time spent in the function, including time spent in functions called by the function.
* ```percall```: the time spent in the function per call, including time spent in functions called by the function.
* ```filename:lineno(function)```: the filename, line number and function name.

There will normally be a number of functions which are not functions you are written or explicitly called. These are often called as part of how Python internally executes your code. They are normally not very consequential in terms of run-time and can often be ignored.

### Profiling By Line

Sometimes, knowing that a code spends a lot of time executing a function doesn't provide enough granularity to know exactly which operations are costly. In these cases, there are several options for find out how much time the code spends executing each line. One example is the  [```line_profiler```](https://pypi.org/project/line-profiler/) module.

The first two lines of the following cell install and load this module. The final line uses the line profiler to call the function ```function_to_test``` with a value of $10^6$ passed to the ```upper_value``` argument. The function ```is_even``` is specified as the function to be profiled.

Running the below cell calls ```function_to_test``` and reports the number of times each line is executed, the length of each execution and the total and fraction of time spent on each line of the function being profiled.

In [ ]:
#Install the relevant module
%pip install line_profiler

#Load the module
%load_ext line_profiler

def is_even(value):
  if value%2 == 0:
    return True
  else:
    return False

def halve(value):
  return value / 2

def function_to_test(upper_value):
  result = 0
  for i in range(int(upper_value) +1):
    if is_even(i):
      result=result + halve(i)

  print("The result is: ", result)

#"lprun" invokes the line profiler
#We specify "is_even" as the function we want to profile
#"function_to_test(1e6)" specifies the command we want to be executed
#This allows the performance of a function to be tested in a specified context
%lprun -f is_even function_to_test(1e6)

In Colab, the results will be presented in a pop-up. In Anaconda or VS Code, the results will be displayed in output below the cell. The output shows the ```timer_unit``` (we'll need this in a minute), the total time and, for each line of the profiled function:
* ```Hits```: the number of times the line was executed.
* ```Time```: the number of timer units spent on that line in total. You can find the total time by multiplying this by the "Timer unit" at the top of the output.
* ```Per Hit```: the average number of timer units spent on that line per execution.
* ```% Time```: the percentage of the total time spent on that line.
* ```Line Contents```: the contents of the line.

### The Effects of Profiling on Run-time

One important note when using any profiling technique is that it can impact the run-time and memory usage of the code. This is because tracking various aspects of performance can actually impact performance as more information needs to be collected. In the example below we run the same code with and without profiling to show the difference in run-time.

In [ ]:
#Install the relevant module
%pip install line_profiler

#Load the module
%load_ext line_profiler

def a_long_loop():
  total = 0
  for i in range(1000000):
    total = total + i

# Time the code without the profiler
start_time = time.time()
a_long_loop()
end_time = time.time()
time_no_profiler = end_time - start_time

# Time the code with the profiler
start_time = time.time()
%lprun -f a_long_loop a_long_loop()
end_time = time.time()
time_with_profiler = end_time - start_time

print("Time without profiler: ", time_no_profiler)
print("Time with profiler: ", time_with_profiler)

This occurs because the profiler is running alongside your code, collecting data. This can significantly increase run-time, so do not be surprised if the code runs slower when profiling.

### Exercise

The code in the cell below has a particular bottleneck that determines the bulk of the runtime of the code.

Examine the code by eye and try to understand how it works, where the bottleneck might lie and what you might do to optimise it. Don't run or change the code at this point.

In [ ]:
# The goal of this code is to print either "X is is prime" or "X is not prime" for every value of X between 1 and 1000

import math

# This method checks if "value" is prime and returns True if it is and False if it isn't
def check_prime(value):
  #Initially assume the value is prime
  prime=True

  # Loop over the values which value may be divisible by and check if it's a multiple of any of them
  for i in range(2, int(math.sqrt(value))+1):
    if value%i==0:
      # If value is divisible by i, value%i will be zero, so set prime to false
      prime=False

  # Now we've checked if value is divisible by any of the values it might be divisible by, return prime
  return prime

# This function prints a message dependent on whether the given value is prime or not
def print_value_prime(value):
  if check_prime(value):
    print(value, " is prime")
  else:
    print(value, " is not prime")

def print_primes(max_value):
# Loop over all numbers between 1 and max_value, printing if they're prime or not
  for i in range(1, max_value+1):
    print_value_prime(i)

#Print whether values between 1 and 1000 are prime
print_primes(1000)

Use ```cProfile```  to work out which function this bottleneck lies in, then use ```line_profiler``` to work out which line(s) the bottleneck lies in. Does this match where you thought it might be by eye? Would it be worth performing the optimisations you came up with earlier?

In [ ]:
#@title

import math
import cProfile

%pip install line_profiler
%load_ext line_profiler

# This method checks if "value" is prime and returns True if it is and False if it isn't
def check_prime(value):
  #Initially assume the value is prime
  prime=True

  # Loop over the values which value may be divisible by and check if it's a multiple of any of them
  for i in range(2, int(math.sqrt(value))+1):
    if value%i==0:
      # If value is divisible by i, value%i will be zero, so set prime to false
      prime=False

  # Now we've checked if value is divisible by any of the values it might be divisible by, return prime
  return prime

# This function prints a message dependent on whether the given value is prime or not
def print_value_prime(value):
  if check_prime(value):
    print(value, " is prime")
  else:
    print(value, " is not prime")

def print_primes(max_value):
# Loop over all numbers between 1 and max_value, printing if they're prime or not
  for i in range(1, max_value+1):
    print_value_prime(i)

#Print whether values between 1 and 1000 are prime
cProfile.run("print_primes(1000)")
#%lprun -f print_value_prime print_primes(1000)

## Memory Profiling



It's also possible to profile the memory which is used by the code. The amount of memory used will change over time as the code runs and data is created and discarded. One meaningful measurement which can be made is the difference in the amount of memory used at two different points in the program. If the first point is the start of the program then this will give a good approximation of how much memory is in use at the second point. Often it will be desirable for the second point to be when the maximum memory is in use and this may take some experimentation to find.

One example of a memory profiler in Python is [```pympler```](https://pympler.readthedocs.io/en/latest/index.html#). An example is found below:

In [ ]:
#Install pympler using pip
%pip install pympler

#Import the relevant parts of pympler
from pympler import summary, muppy

#Get the first measure of the objects in memory
sum1 = muppy.get_objects()

#Create some new objects in memory
list1=[]

for i in range(100):
  new_dict = {}
  for j in range(100):
    new_dict[j] = [1000+i*100+j]
  
  list1.append(new_dict)

#Get the second measure of the objects in memory
sum2 = muppy.get_objects()

#Print a comparison the two measure of the objects in memory
summary.print_(summary.get_diff(summary.summarize(sum1), summary.summarize(sum2)))

The summary printed shows the change in the number of different types of object stored in memory and the change in the amount of memory allocated to each type of object. In the example above, we have created a list containing 100 dictionaries, each containing 100 lists with each list containing one unique integer. Note that there are also a small number of other values created. These may be created due to processes happening "under the hood". Given their small contribution to the total number of values stored, they're not very significant.